In [46]:
import requests
import pandas as pd
import re

from bs4 import BeautifulSoup
from tqdm import tqdm

In [47]:
url_base = 'https://www.zapimoveis.com.br/aluguel/apartamentos/pr+londrina/'

In [48]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}


In [110]:
page_num = 1

# lista para armazenar os informações
property_list = []


while True:
    
    url = f'{url_base}?onde=,Paraná,Londrina,,,,,city,BR>Parana>NULL>Londrina,-23.319731,-51.166201,%2Faluguel%2Fimoveis%2Fpr%2Blondrina%2F&transacao=Aluguel&tipo=Imóvel%20usado&tipos=apartamento_residencial&pagina={page_num}'

    response = requests.get(url, headers=headers)

    if response.status_code == 200:

        # Analisar o HTML retornado usando o BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Encontre as informações relevantes na página usando as tags HTML apropriadas
        properties = soup.find_all('div', class_='card-container')

        if len(properties) == 0:
            break

        # Extrair as informações de cada imóvel encontrado
        for prop in tqdm(properties):
            # Encontre o preço, área e número de quartos do imóvel
            preco_tag = prop.find('p', class_='simple-card__price')
            preco = preco_tag.text if preco_tag else ''

            condominio_tag = prop.find('span', class_='card-price__value')
            condominio = condominio_tag.text if condominio_tag else ''

            endereco_tag = prop.find('h2', class_='simple-card__address')
            endereco = endereco_tag.text if endereco_tag else ''
            
            area_tag = prop.find('span', itemprop='floorSize')
            area = area_tag.text if area_tag else ''

            quartos_tag = prop.find('span', itemprop='numberOfRooms')
            quartos = quartos_tag.text if quartos_tag else ''
            
            banheiros_tag = prop.find('span', itemprop='numberOfBathroomsTotal')
            banheiros = banheiros_tag.text if banheiros_tag else ''

            garage_tag = prop.find('li', class_='feature__item text-small js-parking-spaces')
            garagem = garage_tag.text if garage_tag else ''

            # Imprima as informações do imóvel
            #print(f'Preço: {preco}\nCondomínio: {condominio}\nEndereço: {endereco}\nÁrea: {area}\nQuartos: {quartos}\nBanheiros: {banheiros}')

            property_dict = {'Preço': preco, 'Condomínio': condominio, 'Endereço': endereco, 'Area': area, 'Quartos': quartos, 'Banheiros': banheiros, 'Vagas_garagem': garagem}
            property_list.append(property_dict)

        page_num += 1
    
    else:
        print(f'Erro ao carregar a página {page_num}')
        break

100%|██████████| 46/46 [00:00<00:00, 1008.71it/s]


In [125]:
df_apartamentos = pd.DataFrame(property_list)

df_apartamentos

,Preço,Condomínio,Endereço,Area,Quartos,Banheiros,Vagas_garagem
0,\n R$ 2.900\n /mês\n,R$ 485,"\n Rua José Monteiro de Mello, Gleb...",\n 81 m²\n,\n 3 \n,\n 2 \n,parking\n 1 \n
1,\n R$ 3.390\n /mês\n,R$ 610,"\n Rua Antônio Pisicchio, Gleba Pal...",\n 85 m²\n,\n 3 \n,\n 2 \n,parking\n 2 \n
2,\n R$ 2.600\n /mês\n,R$ 334,\n Avenida José Gabriel de Oliveira...,\n 70 m²\n,\n 3 \n,\n 2 \n,parking\n 1 \n
3,\n R$ 1.400\n /mês\n,R$ 450,"\n Vale dos Tucanos, Londrina\n ...",\n 56 m²\n,\n 2 \n,\n 1 \n,parking\n 1 \n
4,\n R$ 1.600\n /mês\n,R$ 360,"\n Gleba Fazenda Palhano, Londrina\...",\n 54 m²\n,\n 2 \n,\n 1 \n,parking\n 1 \n
...,...,...,...,...,...,...,...
1541,\n R$ 17.000\n /mês\n,R$ 3.000,"\n Rua Montevidéu, Guanabara\n ...",\n 378 m²\n,\n 3 \n,\n 4 \n,parking\n 3 \n
1542,\n R$ 5.500\n /mês\n,R$ 2.700,"\n Rua Santos, Centro\n",\n 600 m²\n,\n 4 \n,\n 8 \n,parking\n 4 \n
1543,\n R$ 11.000\n /mês\n,R$ 5.000,"\n Gleba Fazenda Palhano, Londrina\...",\n 446 m²\n,\n 4 \n,\n 6 \n,parking\n 4 \n
1544,\n R$ 5.500\n /mês\n,R$ 3.000,"\n Centro, Londrina\n",\n 769 m²\n,\n 5 \n,\n 8 \n,parking\n 4 \n


In [126]:
def limpa_dataframe(df):
    for col in df.columns:
        if col != 'Endereço':
            df[col] = pd.to_numeric(df[col].apply(lambda x: re.sub('[^0-9]', '', str(x))))
        else:
            df[col] = df[col].apply(lambda x: re.sub(r'[\n]', ' ', str(x)))
            df[['Rua', 'Bairro']] = df['Endereço'].str.split(',', n=1, expand=True)
            df['Rua'] = df['Rua'].str.strip()
            df['Bairro'] = df['Bairro'].str.strip()

            mask = ~(df['Rua'].str.startswith('Rua') | df['Rua'].str.startswith('Avenida'))
            df.loc[mask, 'Bairro'] = df.loc[mask, 'Rua']
            df.loc[mask, 'Rua'] = ''

            df['Bairro'] = df['Bairro'].apply(lambda x: '' if x == 'Londrina' else x)
    
    df['Vagas_garagem'] = df['Vagas_garagem'].fillna(0)
    df['Vagas_garagem'] = df['Vagas_garagem'].astype(int)

    df.insert(0, 'Tipo', 'apartamento')

    df.drop(['Endereço'], axis=1, inplace=True)

    return df


limpa_dataframe(df_apartamentos)

,Tipo,Preço,Condomínio,Area,Quartos,Banheiros,Vagas_garagem,Rua,Bairro
0,apartamento,2900,485.0,81,3,2,1,Rua José Monteiro de Mello,Gleba Palhano
1,apartamento,3390,610.0,85,3,2,2,Rua Antônio Pisicchio,Gleba Palhano
2,apartamento,2600,334.0,70,3,2,1,Avenida José Gabriel de Oliveira,Aurora
3,apartamento,1400,450.0,56,2,1,1,,Vale dos Tucanos
4,apartamento,1600,360.0,54,2,1,1,,Gleba Fazenda Palhano
...,...,...,...,...,...,...,...,...,...
1541,apartamento,17000,3000.0,378,3,4,3,Rua Montevidéu,Guanabara
1542,apartamento,5500,2700.0,600,4,8,4,Rua Santos,Centro
1543,apartamento,11000,5000.0,446,4,6,4,,Gleba Fazenda Palhano
1544,apartamento,5500,3000.0,769,5,8,4,,Centro


In [122]:
df_apartamentos.dtypes

Preço              int64
Condomínio       float64
Area               int64
Quartos            int64
Banheiros          int64
Vagas_garagem      int32
Rua               object
Bairro            object
dtype: object

In [123]:
df_apartamentos[(df_apartamentos['Preço']>=1500) & (df_apartamentos['Preço']<=2000) & (df_apartamentos['Quartos']==3) & (df_apartamentos['Banheiros']==2)]

,Preço,Condomínio,Area,Quartos,Banheiros,Vagas_garagem,Rua,Bairro
17,1800,604.0,138,3,2,1,,Vila Larsen 1
27,1700,380.0,69,3,2,1,Rua Ulrico Zuínglio,Gleba Palhano
29,1500,270.0,80,3,2,2,Avenida Inglaterra,Igapo
40,1700,515.0,86,3,2,1,Rua Brasil,Centro
43,1500,500.0,127,3,2,1,,Dom Bosco
...,...,...,...,...,...,...,...,...
1306,1600,550.0,170,3,2,1,,Centro
1320,1890,290.0,69,3,2,1,,Terra Bonita
1327,1950,480.0,107,3,2,1,,Centro
1393,1600,800.0,97,3,2,0,,Gleba Palhano


In [124]:
df_apartamentos[df_apartamentos['Vagas_garagem']==0]

,Preço,Condomínio,Area,Quartos,Banheiros,Vagas_garagem,Rua,Bairro
10,1300,400.0,50,2,1,0,,Judith
11,1200,250.0,50,2,1,0,Rua Carmela Dutra,Jardim Morumbi
13,1000,300.0,44,2,1,0,,Jardim Sao Paulo II
26,1100,480.0,46,2,1,0,,Vale dos Tucanos
57,780,214.0,110,3,2,0,,Praça Gabriel Martins
...,...,...,...,...,...,...,...,...
1507,1200,1300.0,114,3,3,0,,Centro
1518,650,950.0,160,3,2,0,Rua Maranhão,Centro
1529,10000,2000.0,370,4,4,0,,Gleba Fazenda Palhano
1534,11000,4999.0,445,4,7,0,,Gleba Palhano
